# Reverse Time Integration

In [1]:
using Plots
using Random
using Statistics
using CUDA

In [2]:
mutable struct Particles
    n :: Int
    pos :: AbstractArray{Float64, 2}
    vel :: AbstractArray{Float64, 2}
end

In [15]:
function gravity(particles::Particles) :: AbstractArray{Float64, 2}
    vec_x = particles.pos[1, :] .- particles.pos[1, :]'
    vec_y = particles.pos[2, :] .- particles.pos[2, :]'
    vec_z = particles.pos[3, :] .- particles.pos[3, :]'

    # softening ~ average spacing and ^2
    # maybe look into softening methods
    r_sq = (vec_x.^2 + vec_y.^2 + vec_z.^2) .+ 1e-5

    # ungebunden -> kinetic energy higher than potential energy by gravity

    force_x = -vec_x ./ r_sq
    force_y = -vec_y ./ r_sq
    force_z = -vec_z ./ r_sq

    force_x = sum(force_x, dims=2)
    force_y = sum(force_y, dims=2)
    force_z = sum(force_z, dims=2)

    force = [force_x force_y force_z]

    return transpose(force)
end

gravity (generic function with 1 method)

In [4]:
# kick drift kick / leap frog
function kdk(particles::Particles, dt::Float64)
    # compute a(t)
    force = gravity(particles)
    # v(t + dt/2) = v(t) + a(t) * dt/2
    particles.vel .+= force .* dt / 2
    # x(t + dt) = x(t) + v(t + dt/2) * dt
    particles.pos .+= particles.vel .* dt
    # a(t + dt)
    force = gravity(particles)
    # v(t + dt) = v(t + dt/2) + a(t + dt) * dt/2
    particles.vel .+= force .* dt / 2
end

# reverse time integration
function rev_kdk(particles::Particles, dt::Float64)
    # compute a(t)
    force = gravity(particles)
    # v(t - dt/2) = v(t) - a(t) * dt/2
    particles.vel .-= force .* dt / 2
    # compute x(t - dt)
    particles.pos .-= particles.vel .* dt
    # a(t - dt)
    force = gravity(particles)
    # v(t - dt) = v(t - dt/2) - a(t - dt) * dt/2
    particles.vel .-= force .* dt / 2
end

rev_kdk (generic function with 1 method)

In [16]:

function compute_reverse_error(
    n_particles::Int, n_steps::Int, n_dims::Int, dt::Float64    ) :: Float64

    pos = rand(Float64, (n_dims, n_particles)).-0.5
    vel = rand(Float64, (n_dims, n_particles)).-0.5

    pos_ic = Array(pos)

    particles = Particles(n_particles, pos, vel)

    # do the forward simulation
    for i in 1:n_steps
        kdk(particles, dt)
    end

    for i in 1:n_steps-1
        rev_kdk(particles, dt)
    end

    pos_end = Array(particles.pos)

    # compute the error
    error = sum((pos_ic .- pos_end).^2) / n_particles
    return error

end

compute_reverse_error (generic function with 1 method)

In [6]:
n_measures = 8
n_shots = 5

5

In [17]:
# lets see if the error increases with higher dt

errors_dt = zeros(n_measures)
errors_dt_std = zeros(n_measures)

# log scale range of dts
dts = 10.0.^(range(-3,stop=-1,length=n_measures))

for i in 1:n_measures
    errors = zeros(n_shots)
    n_steps = Int(floor(1.0 / dts[i]))
    for j in 1:n_shots
        errors[j] += compute_reverse_error(100, n_steps, 3, dts[i])
    end
    errors_dt_std[i] = std(errors)
    errors_dt[i] = mean(errors)
end

In [18]:
# lets see if it also increase with more particle
errors_n = zeros(n_measures)
errors_n_std = zeros(n_measures)
# range of num_particles
num_particles = range(10, stop=400, length=n_measures)

for i in 1:n_measures
    errors = zeros(n_shots)
    for j in 1:n_shots
        Random.seed!(j)
        errors[j] += compute_reverse_error(Int(floor(num_particles[i])), 40, 3, 1e-2)
    end
    errors_n_std[i] = std(errors)
    errors_n[i] = mean(errors)
end


In [21]:
# lets see if it also increase with more time steps
errors_steps = zeros(n_measures)
errors_steps_std = zeros(n_measures)

# range of num_particles
num_steps = 10.0.^(range(2,stop=3,length=n_measures))


for i in 1:n_measures
    errors = zeros(n_shots)
    for j in 1:n_shots
        Random.seed!(j)
        errors[j] += compute_reverse_error(100, Int(floor(num_steps[i])), 3, 1e-2)
    end
    errors_steps_std[i] = std(errors)
    errors_steps[i] = mean(errors)
end

In [22]:
p1 = plot(dts, errors_dt, xaxis=:log, yaxis=:log, label="dt", xlabel="dt", ylabel="error", title="Error vs dt")
p2 = plot(num_particles, errors_n, xaxis=:log, yaxis=:log, label="num_particles", xlabel="num_particles", ylabel="error", title="Error vs num_particles")
p3 = plot(num_steps, errors_steps, xaxis=:log, yaxis=:log, label="num_steps", xlabel="num_steps", ylabel="error", title="Error vs num_steps")
# set text size

plot(p1, p2, p3, layout=(1,3), legend=:topleft, size=(1000, 300), margin=5Plots.mm)
savefig("error.png")


"c:\\Users\\andri\\projects\\msc-thesis\\experiments\\error.png"